## Carga de datos

Antes de poder manipular los diferentes datos, primero es necesário
cargarlos en el sistema

In [11]:
import polars as pl
import numpy as np 
from pathlib import Path
from sklearn.metrics.pairwise import haversine_distances

In [12]:
# Directório de archivos
data_folder = Path("../data")

# Diccionario de archivos
files = {
    "train"              : "train.csv",
    "gas_prices"         : "gas_prices.csv",
    "client"             : "client.csv",
    "electricity_prices" : "electricity_prices.csv",
    "forecast_weather"   : "forecast_weather.csv",
    "historical_weather" : "historical_weather.csv",
    "weather_station"    : "weather_station_to_county_mapping.csv"
}

In [13]:
# Se crea un diccionario de DataFrames para acceder facilmente a todos
# y poder iterar sobre ellos si es necesário
dfs: dict[str, pl.DataFrame] = {}

for key, filename in files.items():
    filepath = data_folder / filename
    try:
        df = pl.read_csv(filepath)
        dfs[key] = df
        print(f"Archivo {key} cargado con exito.")
    except:
        print(f"No pudo cargarse el archivo {key}.")
    

Archivo train cargado con exito.
Archivo gas_prices cargado con exito.
Archivo client cargado con exito.
Archivo electricity_prices cargado con exito.
Archivo forecast_weather cargado con exito.
Archivo historical_weather cargado con exito.
Archivo weather_station cargado con exito.


## Asignación de longitud/latitud a un condado especifico

Con el fin de poder unir los csv de "train.csv" y "forecast_weather.csv"
es necesario poder asignar a cada longitud y latitud, su condado
correspondiente.

Para ello se utilizará el archivo "weather_station_to_county_mapping" con datos que asignan latitudes y longitudes a su correspondiente condado.

### Visualización de los datos

In [14]:
class DataProcessor:
    def __init__(self):
        self.gas_join = ["date"]
        self.ep_join = ["datetime"]
        self.hw_join = [ "county", "datetime"]
        self.client_join = ["product_type", "county", "is_business", "date"]
        self.lat_lon = ["latitude", "longitude"]

    def add_counties(self, df: pl.DataFrame, stations: pl.DataFrame):
        # Se seleccionan los datos requeridos

        coords = df.select(
            pl.col("latitude", "longitude")
        ).unique()

        coords = coords.with_columns(
            pl.col("latitude").radians().alias("lat_rad"),
            pl.col("longitude").radians().alias("lon_rad")
        )

        wthr_stations = stations.drop_nulls()
        wthr_stations = wthr_stations[
            "latitude", "longitude", "county"
        ].with_columns(
                pl.col("latitude", "longitude").radians()
        ).to_numpy()

        coords_arr = coords.to_numpy()

        counties = np.array([], np.int32)
        # Se itera para obtener la menor distancia entre puntos 
        # y asignar el condado
        for coord in coords_arr[:, 2:4]:
            # Cada coordenada tiene una dist minima 
            # y un condado asignado
            min_dist = float("inf")
            cnty = -1

            station_zip = zip(wthr_stations[:, :2], wthr_stations[:, 2])

            for lat_lon, county in station_zip:
                dist = haversine_distances([coord, lat_lon])[0, 1]

                if dist < min_dist:
                    cnty = county
                    min_dist = dist

            # Se añade el condado a la lista
            counties = np.append(
                counties,
                int(cnty)
            )
        
        # Se asigna los condados a cada coordenada y se devuelve el resultado
        result = coords.with_columns(
            pl.lit(counties).alias("county")
        )

        result = df.join(
            result.select(["latitude", "longitude", "county"]),
            on=["latitude", "longitude"],
            how="left"
        )

        return result

    def change_names(self, df: pl.DataFrame, suffix, no_change):
        renamed_df = df.rename(
            lambda col: col + suffix if col not in no_change else col
        )
        return renamed_df
    
    def to_datetime(self, df: pl.DataFrame, col: str):
        # Cast str to Datetime
        result = df.with_columns(
            pl.col(col).str.strptime(pl.Datetime)
        )
        return result

    def get_gas_features(self, gas_df: pl.DataFrame):
        df = gas_df.with_columns(
            ((pl.col("lowest_price_per_mwh") + 
              pl.col("highest_price_per_mwh")) / 2).alias("mean_price_per_mwh_24h")
        )

        df = df.with_columns(pl.col("origin_date").str.strptime(pl.Date))
        df = df.rename({"origin_date":"date"})

        return self.change_names(df, "_gas", self.gas_join)
    
    def get_client_features(self, client_df: pl.DataFrame):
        df = client_df.with_columns(
            pl.col("date").str.strptime(pl.Date)
        )
        return self.change_names(df, "_client", self.client_join)
    
    def get_hw_means(self, data: pl.DataFrame) -> pl.DataFrame:
        data = data.group_by(
        ["datetime", "county"], maintain_order=True
        ).agg(pl.all().mean())
        return data

    def get_hist_weather_features(self, hw_df: pl.DataFrame, ws_df: pl.DataFrame):
        df = self.to_datetime(hw_df, "datetime")

        df = self.add_counties(df, ws_df)
        df = self.get_hw_means(df)
        df = self.change_names(df, "_hw", self.hw_join)

        return df
    
    def get_electricity_features(self, ep_df: pl.DataFrame):
        # str to datetime
        df = self.to_datetime(ep_df, "origin_date")

        df = df.rename({"origin_date":"datetime"})
        df = df.rename({"euros_per_mwh":"euros_per_mwh_24h"})

        # Change names
        df = self.change_names(df, "_ep", self.ep_join)

        return df
    
    def get_data_features(self, data: pl.DataFrame):
        
        date = "datetime"
        # Cast to Datetime
        df = self.to_datetime(data, "datetime")

        df = df.with_columns([
            # Date
            pl.col(date).dt.truncate("1d").cast(pl.Date).alias("date"),

            # Year
            pl.col(date).dt.year().alias("year"),

            # Quarter
            pl.col(date).dt.quarter().alias("quarter"),

            # Month
            pl.col(date).dt.month().alias("month"),

            # Week
            pl.col(date).dt.week().alias("week"),

            # Hour
            pl.col(date).dt.hour().alias("hour"),

            # Day of year
            pl.col(date).dt.ordinal_day().alias("day_of_year"),

            # Day of month
            pl.col(date).dt.day().alias("day_of_month"),

            # Day of week
            pl.col(date).dt.weekday().alias("day_of_week")
        ])

        return df
    
    def __call__(self, df_dict: dict):

        # Features
        data = self.get_data_features(df_dict["train"])
        gas_p = self.get_gas_features(df_dict["gas_prices"])
        el_p = self.get_electricity_features(df_dict["electricity_prices"])
        client = self.get_client_features(df_dict["client"])
        hist_w = self.get_hist_weather_features(
            df_dict["historical_weather"],
            df_dict["weather_station"]
        )

        # Joins
        final_data = data.join(client, on=self.client_join, how="left")
        final_data = final_data.join(gas_p, on=self.gas_join, how="left")
        final_data = final_data.join(el_p, on=self.ep_join, how="left")
        final_data = final_data.join(hist_w, on=self.hw_join, how="left")

        return final_data

In [15]:
dp = DataProcessor()
res = dp(dfs)

In [16]:
res

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,data_block_id_client,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,data_block_id_gas,mean_price_per_mwh_24h_gas,forecast_date_ep,euros_per_mwh_24h_ep,data_block_id_ep,temperature_hw,dewpoint_hw,rain_hw,snowfall_hw,surface_pressure_hw,cloudcover_total_hw,cloudcover_low_hw,cloudcover_mid_hw,cloudcover_high_hw,windspeed_10m_hw,winddirection_10m_hw,shortwave_radiation_hw,direct_solar_radiation_hw,diffuse_radiation_hw,latitude_hw,longitude_hw,data_block_id_hw
i64,i64,i64,f64,i64,datetime[μs],i64,i64,i64,date,i32,i8,i8,i8,i8,i16,i8,i8,i64,f64,i64,str,f64,f64,i64,f64,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,2021,3,9,35,0,244,1,3,108,952.89,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,1,96.59,1,2021-09-01 00:00:00,0,1,0,2021-09-01,2021,3,9,35,0,244,1,3,108,952.89,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,2,0.0,0,2021-09-01 00:00:00,0,2,1,2021-09-01,2021,3,9,35,0,244,1,3,17,166.4,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,2021,3,9,35,0,244,1,3,17,166.4,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,2021,3,9,35,0,244,1,3,688,7207.88,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31,2023,2,5,22,23,151,31,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
15,1,1,0.0,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31,2023,2,5,22,23,151,31,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31,2023,2,5,22,23,151,31,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [17]:
res.describe()

statistic,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,data_block_id_client,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,data_block_id_gas,mean_price_per_mwh_24h_gas,forecast_date_ep,euros_per_mwh_24h_ep,data_block_id_ep,temperature_hw,dewpoint_hw,rain_hw,snowfall_hw,surface_pressure_hw,cloudcover_total_hw,cloudcover_low_hw,cloudcover_mid_hw,cloudcover_high_hw,windspeed_10m_hw,winddirection_10m_hw,shortwave_radiation_hw,direct_solar_radiation_hw,diffuse_radiation_hw,latitude_hw,longitude_hw,data_block_id_hw
str,f64,f64,f64,f64,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2.018352e6,2.018352e6,2.018352e6,2.017824e6,2.018352e6,"""2018352""",2.018352e6,2.018352e6,2.018352e6,"""2018352""",2.018352e6,2.018352e6,2.018352e6,2.018352e6,2.018352e6,2.018352e6,2.018352e6,2.018352e6,2.012112e6,2.012112e6,2.012112e6,"""2012112""",2.012112e6,2.012112e6,2.012112e6,2.012112e6,"""2011846""",2.011846e6,2.011846e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6,1.982992e6
"""null_count""",0.0,0.0,0.0,528.0,0.0,"""0""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6240.0,6240.0,6240.0,"""6240""",6240.0,6240.0,6240.0,6240.0,"""6506""",6506.0,6506.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0,35360.0
"""mean""",7.297034,0.536826,1.898927,274.85556,0.5,"""2022-07-20 08:29:25.326167""",321.874599,1009175.5,33.045376,"""2022-07-19 20:59:25.326000""",2022.056838,2.478418,6.430617,26.127518,11.5,180.128564,15.701562,3.998121,73.345118,1450.771451,322.898876,null,95.526009,108.385728,322.898876,101.955869,null,157.508343,322.890185,5.446717,1.918643,0.048332,0.016752,1007.951681,62.020768,47.921299,35.125868,36.109873,4.467575,198.422204,105.456573,62.931326,42.525247,58.635229,25.548547,322.716218
"""std""",4.78099,0.498642,1.081766,909.502378,0.5,null,182.634314,582648.179597,19.590594,null,0.645235,1.178726,3.664891,15.934816,6.922188,111.951252,8.78617,1.998947,144.062707,2422.20483,182.073597,null,47.569832,54.742018,182.073597,50.914938,null,121.416689,182.072039,8.169059,7.238676,0.17523,0.069741,12.502175,35.938704,38.986774,36.442652,39.263737,2.045175,82.540755,175.167455,127.936947,61.521575,0.58859,1.63161,182.178293
"""min""",0.0,0.0,0.0,0.0,0.0,"""2021-09-01 00:00:00""",0.0,0.0,0.0,"""2021-09-01""",2021.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0,5.5,2.0,"""2021-09-02""",28.1,34.0,2.0,31.1,"""2021-09-02 00:00:00""",-10.06,2.0,-20.533333,-23.366667,0.0,0.0,952.9,0.0,0.0,0.0,0.0,0.166667,1.0,0.0,0.0,0.0,57.78,22.144444,1.0
"""25%""",3.0,0.0,1.0,0.378,0.0,"""2022-02-14 04:00:00""",166.0,504588.0,16.0,"""2022-02-14""",2022.0,1.0,3.0,12.0,6.0,80.0,8.0,2.0,13.0,321.9,167.0,null,60.0,67.67,167.0,65.0,null,85.29,167.0,-0.47,-3.016667,0.0,0.0,1000.75,28.153846,6.333333,0.75,0.0,2.907407,144.0,0.0,0.0,0.0,58.094118,24.533333,167.0
"""50%""",7.0,1.0,2.0,31.133,1.0,"""2022-07-21 10:00:00""",323.0,1.009176e6,33.0,"""2022-07-21""",2022.0,2.0,6.0,24.0,12.0,166.0,16.0,4.0,32.0,645.2,324.0,null,86.0,94.0,324.0,88.895,null,128.79,324.0,4.975,1.44,0.0,0.0,1009.023077,73.166667,44.692308,21.0,15.909091,4.240741,206.833333,6.153846,0.0,4.4,58.725,25.533333,324.0
"""75%""",11.0,1.0,3.0,180.206,1.0,"""2022-12-24 14:00:00""",479.0,1.513763e6,50.0,"""2022-12-24""",2022.0,4.0,10.0,41.0,17.0,285.0,23.0,6.0,70.0,1567.15,480.0,null,109.74,133.0,480.0,120.91,null,199.99,480.0,10.966667,6.8,0.007692,0.0,1016.217647,97.666667,90.333333,68.5,78.833333,5.733333,257.363636,139.0,47.833333,74.733333,59.233333,27.116667,480.0
"""max""",15.0,1.0,3.0

In [29]:
for key in dfs.keys():
    print(key)
    display(dfs[key].describe())

train


statistic,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
str,f64,f64,f64,f64,f64,str,f64,f64,f64
"""count""",2.018352e6,2.018352e6,2.018352e6,2.017824e6,2.018352e6,"""2018352""",2.018352e6,2.018352e6,2.018352e6
"""null_count""",0.0,0.0,0.0,528.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",7.297034,0.536826,1.898927,274.85556,0.5,null,321.874599,1009175.5,33.045376
"""std""",4.78099,0.498642,1.081766,909.502378,0.5,null,182.634314,582648.179597,19.590594
"""min""",0.0,0.0,0.0,0.0,0.0,"""2021-09-01 00:00:00""",0.0,0.0,0.0
"""25%""",3.0,0.0,1.0,0.378,0.0,null,166.0,504588.0,16.0
"""50%""",7.0,1.0,2.0,31.133,1.0,null,323.0,1.009176e6,33.0
"""75%""",11.0,1.0,3.0,180.206,1.0,null,479.0,1.513763e6,50.0
"""max""",15.0,1.0,3.0,15480.274,1.0,"""2023-05-31 23:00:00""",637.0,2.018351e6,68.0


gas_prices


statistic,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
str,str,f64,f64,str,f64
"""count""","""637""",637.0,637.0,"""637""",637.0
"""null_count""","""0""",0.0,0.0,"""0""",0.0
"""mean""",null,95.03675,107.754631,null,319.0
"""std""",null,47.552295,54.743666,null,184.030342
"""min""","""2021-09-01""",28.1,34.0,"""2021-08-31""",1.0
"""25%""",null,60.0,67.53,null,160.0
"""50%""",null,85.21,93.47,null,319.0
"""75%""",null,109.0,130.74,null,478.0
"""max""","""2023-05-30""",250.0,305.0,"""2023-05-29""",637.0


client


...

statistic,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
"""count""",3.424512e6,3.424512e6,"""3424512""",3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,3.424512e6,"""3424512""",3.424512e6,3.42451e6,3.424512e6,3.424512e6
"""null_count""",0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,2.0,0.0,0.0
"""mean""",58.65,24.95,null,24.5,5.743913,2.411946,0.394665,0.434645,0.359069,0.681993,1.255446,0.725011,319.0,null,151.188212,110.764207,0.000025,0.000079
"""std""",0.687386,2.015565,null,13.853401,7.844206,7.121432,0.444043,0.438635,0.420156,0.400963,3.9953,4.223752,183.885861,null,256.506903,187.444373,0.000122,0.000278
"""min""",57.6,21.7,"""2021-09-01 02:00:00""",1.0,-27.499396,-29.683569,0.0,0.0,0.0,0.0,-17.577179,-22.116119,1.0,"""2021-09-01 03:00:00""",-0.773333,-0.325833,-0.000004,-0.000015
"""25%""",58.2,23.2,null,13.0,0.260645,-2.364355,0.0,0.000336,0.0,0.264893,-1.46669,-1.978104,160.0,null,0.0,0.0,0.0,0.0
"""50%""",58.8,25.2,null,25.0,4.872705,1.835596,0.088669,0.230546,0.101196,0.977267,1.468682,0.943322,319.0,null,0.0,0.604444,0.0,0.0
"""75%""",59.1,26.7,null,36.0,11.146387,7.302026,0.978485,0.999459,0.900085,1.0,3.813531,3.507628,478.0,null,212.844444,144.172326,0.0,0.000028
"""max""",59.7,28.2,"""2023-05-30 02:00:00""",48.0,31.810693,23.680566,1.000008,1.000008,1.000008,1.000008,22.573198,19.314369,637.0,"""2023-06-01 02:00:00""",954.422222,848.714444,0.004833,0.016516


historical_weather


statistic,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""1710802""",1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6,1.710802e6
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,5.740968,2.240312,0.04962,0.016049,1009.281515,60.912696,46.685927,34.40698,36.051408,4.849871,197.869419,106.490504,64.452917,42.037587,58.649999,24.949999,319.270778
"""std""",null,8.025647,7.224357,0.207911,0.074629,13.088915,37.769048,40.747598,38.327693,41.358521,2.47545,89.937978,179.944912,133.409951,61.952251,0.687387,2.015564,183.729798
"""min""","""2021-09-01 00:00:00""",-23.7,-25.9,0.0,0.0,942.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.6,21.7,1.0
"""25%""",null,0.0,-2.6,0.0,0.0,1001.5,25.0,3.0,0.0,0.0,3.0,139.0,0.0,0.0,0.0,57.9,23.2,160.0
"""50%""",null,5.1,1.7,0.0,0.0,1010.4,72.0,39.0,16.0,10.0,4.5,208.0,1.0,0.0,1.0,58.5,24.7,319.0
"""75%""",null,11.2,7.2,0.0,0.0,1018.0,100.0,94.0,72.0,85.0,6.277778,263.0,140.0,47.0,74.0,59.1,26.7,478.0
"""max""","""2023-05-30 10:00:00""",32.6,23.8,16.8,2.66,1049.3,100.0,100.0,100.0,100.0,21.75,360.0,849.0,754.0,388.0,59.7,28.2,637.0


weather_station


statistic,county_name,longitude,latitude,county
str,str,f64,f64,f64
"""count""","""49""",112.0,112.0,49.0
"""null_count""","""63""",0.0,0.0,63.0
"""mean""",null,24.95,58.65,7.061224
"""std""",null,2.024623,0.690476,4.870866
"""min""","""Harjumaa""",21.7,57.6,0.0
"""25%""",null,23.2,58.2,3.0
"""50%""",null,25.2,58.8,7.0
"""75%""",null,26.7,59.1,11.0
"""max""","""Võrumaa""",28.2,59.7,15.0


In [18]:
dfs["client"].head()

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,str,i64
1,0,108,952.89,0,"""2021-09-01""",2
2,0,17,166.4,0,"""2021-09-01""",2
3,0,688,7207.88,0,"""2021-09-01""",2
0,0,5,400.0,1,"""2021-09-01""",2
1,0,43,1411.0,1,"""2021-09-01""",2


In [11]:
# Adivinar que hace la variable hours ahead f mean
# No añadir los Targets de horas anteriores

In [31]:
dfs["client"].tail()

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,str,i64
1,15,51,415.6,0,"""2023-05-29""",637
3,15,161,2035.75,0,"""2023-05-29""",637
0,15,15,620.0,1,"""2023-05-29""",637
1,15,20,624.5,1,"""2023-05-29""",637
3,15,55,2188.2,1,"""2023-05-29""",637


In [41]:
display(dfs["train"])
display(dfs["electricity_prices"])

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,0.713,0,"""2021-09-01 00:00:00""",0,0,0
0,0,1,96.59,1,"""2021-09-01 00:00:00""",0,1,0
0,0,2,0.0,0,"""2021-09-01 00:00:00""",0,2,1
0,0,2,17.314,1,"""2021-09-01 00:00:00""",0,3,1
0,0,3,2.904,0,"""2021-09-01 00:00:00""",0,4,2
…,…,…,…,…,…,…,…,…
15,1,0,197.233,1,"""2023-05-31 23:00:00""",637,2018347,64
15,1,1,0.0,0,"""2023-05-31 23:00:00""",637,2018348,59
15,1,1,28.404,1,"""2023-05-31 23:00:00""",637,2018349,59


forecast_date,euros_per_mwh,origin_date,data_block_id
str,f64,str,i64
"""2021-09-01 00:00:00""",92.51,"""2021-08-31 00:00:00""",1
"""2021-09-01 01:00:00""",88.9,"""2021-08-31 01:00:00""",1
"""2021-09-01 02:00:00""",87.35,"""2021-08-31 02:00:00""",1
"""2021-09-01 03:00:00""",86.88,"""2021-08-31 03:00:00""",1
"""2021-09-01 04:00:00""",88.43,"""2021-08-31 04:00:00""",1
…,…,…,…
"""2023-05-30 19:00:00""",82.1,"""2023-05-29 19:00:00""",637
"""2023-05-30 20:00:00""",150.85,"""2023-05-29 20:00:00""",637
"""2023-05-30 21:00:00""",82.1,"""2023-05-29 21:00:00""",637


In [ ]:
for key in dfs.keys():
    print(f"{key}:")
    display(dfs[key].head(1))

train:


county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,0.713,0,"""2021-09-01 00:00:00""",0,0,0


gas_prices:


forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
str,f64,f64,str,i64
"""2021-09-01""",45.23,46.32,"""2021-08-31""",1


client:


...

datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
str,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,i64,f64,f64,f64,f64,f64,f64
"""2021-09-01 00:00:00""",14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.083333,8,0.0,0.0,0.0,57.6,21.7,1.0


county_lon_lats:


,county,longitude,latitude
i64,i64,f64,f64
0,0,24.2,59.1


weather_station:


county_name,longitude,latitude,county
str,f64,f64,i64
null,21.7,57.6,null


In [ ]:
dfs["client"].with_columns(
    pl.col("date").str.strptime(pl.Date)
)
.0

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,date,i64
1,0,108,952.89,0,2021-09-01,2
2,0,17,166.4,0,2021-09-01,2
3,0,688,7207.88,0,2021-09-01,2
0,0,5,400.0,1,2021-09-01,2
1,0,43,1411.0,1,2021-09-01,2
…,…,…,…,…,…,…
1,15,51,415.6,0,2023-05-29,637
3,15,161,2035.75,0,2023-05-29,637
0,15,15,620.0,1,2023-05-29,637


In [17]:
display(dfs["client"].filter(
    (pl.col("product_type") == 1) & 
    (pl.col("county") == 0) & 
    (pl.col("is_business") == 0)
))

product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
i64,i64,i64,f64,i64,str,i64
1,0,108,952.89,0,"""2021-09-01""",2
1,0,108,952.89,0,"""2021-09-02""",3
1,0,108,952.89,0,"""2021-09-03""",4
1,0,108,952.89,0,"""2021-09-04""",5
1,0,108,952.89,0,"""2021-09-05""",6
…,…,…,…,…,…,…
1,0,508,4968.215,0,"""2023-05-25""",633
1,0,507,4960.215,0,"""2023-05-26""",634
1,0,507,4960.215,0,"""2023-05-27""",635


In [34]:
dfs["train"].filter(
    pl.col("data_block_id") == 1
)

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,str,i64,i64,i64
0,0,1,1.687,0,"""2021-09-02 00:00:00""",1,2928,0
0,0,1,109.366,1,"""2021-09-02 00:00:00""",1,2929,0
0,0,2,0.0,0,"""2021-09-02 00:00:00""",1,2930,1
0,0,2,21.008,1,"""2021-09-02 00:00:00""",1,2931,1
0,0,3,1.003,0,"""2021-09-02 00:00:00""",1,2932,2
…,…,…,…,…,…,…,…,…
15,0,3,59.302,1,"""2021-09-02 23:00:00""",1,5851,58
15,1,1,0.0,0,"""2021-09-02 23:00:00""",1,5852,59
15,1,1,21.756,1,"""2021-09-02 23:00:00""",1,5853,59


In [13]:
dfs["forecast_weather"].filter(
    (pl.col("latitude") == 57.6) &
    (pl.col("longitude") == 21.7) &
    (pl.col("origin_datetime") == "2021-09-01 02:00:00")
).select(
    ["latitude", "longitude", "hours_ahead", "origin_datetime"]
).group_by("origin_datetime").agg(pl.col("hours_ahead").mean())

origin_datetime,hours_ahead
str,f64
"""2021-09-01 02:00:00""",24.5


In [14]:
agg_columns = [col for col in dfs["forecast_weather"].columns 
               if col not in ["latitude", "longitude"] + 
               ["forecast_datetime", 'data_block_id']]
agg_dict = {agg_col: ["mean"] for agg_col in agg_columns}

dfs["forecast_weather"].group_by(["forecast_datetime", 'data_block_id']).agg(
    pl.mean(agg_columns)
)

forecast_datetime,data_block_id,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2021-11-05 02:00:00""",65,null,25.0,7.517538,6.483776,0.374459,0.788241,0.930931,0.999334,-5.107638,2.407442,0.0,0.0,0.0,0.000828
"""2023-01-13 01:00:00""",498,null,48.0,2.547195,1.822729,0.230212,0.996794,0.714138,0.99985,1.465805,6.375389,0.0,0.0,0.000116,0.000515
"""2022-11-07 06:00:00""",432,null,29.0,7.805334,6.237741,0.9839,0.646853,0.686397,0.9998,3.204765,5.962434,0.0,0.0,0.0,0.000066
"""2021-11-26 22:00:00""",86,null,45.0,0.462178,-1.622852,0.68024,0.548291,0.894366,0.961348,-3.388509,-1.414764,0.0,0.0,0.000109,0.000138
"""2023-04-06 15:00:00""",583,null,13.0,7.781676,3.271873,0.371023,0.161743,0.121847,0.429985,-3.997157,-1.128954,631.914921,456.694643,0.0,6.4926e-7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023-02-25 21:00:00""",542,null,44.0,-3.086759,-5.321074,0.739758,0.942859,0.748293,0.955175,-3.73673,-7.943222,0.0,0.0,0.000057,0.000057
"""2023-02-27 15:00:00""",544,null,38.0,-2.958223,-5.219591,0.231138,0.465095,0.008772,0.525808,3.614254,1.136308,414.168333,183.839881,1.5966e-7,7.8763e-8
"""2022-06-18 06:00:00""",290,null,28.0,14.00735,12.099392,0.999698,0.017041,0.60023,0.999998,1.996138,3.770625,77.048254,108.461587,0.0,0.000002


In [15]:
dfs["forecast_weather"].head()


latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
f64,f64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,str,f64,f64,f64,f64
57.6,21.7,"""2021-09-01 02:00:00""",1,15.655786,11.553613,0.904816,0.019714,0.0,0.905899,-0.411328,-9.106137,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,22.2,"""2021-09-01 02:00:00""",1,13.003931,10.689844,0.886322,0.004456,0.0,0.886658,0.206347,-5.355405,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,22.7,"""2021-09-01 02:00:00""",1,14.206567,11.671777,0.729034,0.005615,0.0,0.730499,1.451587,-7.417905,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,23.2,"""2021-09-01 02:00:00""",1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0
57.6,23.7,"""2021-09-01 02:00:00""",1,15.293848,12.458887,0.102875,0.088074,0.000015,0.17659,1.268481,-8.975766,1,"""2021-09-01 03:00:00""",0.0,0.0,0.0,0.0


In [32]:
result = dfs["train"].with_columns(
    pl.col("datetime").str.strptime(
        pl.Datetime, "%Y-%m-%d %H:%M:%S"
    )
)

In [33]:
result

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
i64,i64,i64,f64,i64,datetime[μs],i64,i64,i64
0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
0,0,1,96.59,1,2021-09-01 00:00:00,0,1,0
0,0,2,0.0,0,2021-09-01 00:00:00,0,2,1
0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
…,…,…,…,…,…,…,…,…
15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
15,1,1,0.0,0,2023-05-31 23:00:00,637,2018348,59
15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59


In [42]:
result = result.with_columns([
    pl.col("datetime").dt.truncate("1d").cast(pl.Date).alias("date"),
    pl.col("datetime").dt.year().alias("year")
])

In [24]:
dfa = pl.DataFrame({
    "A":[1, 3, 4, 3, 2],
    "B":["uno", "tres", "cuatro", "tres", "dos"]
})
dfa

A,B
i64,str
1,"""uno"""
3,"""tres"""
4,"""cuatro"""
3,"""tres"""
2,"""dos"""


In [25]:
dfb = pl.DataFrame({
    "A":[1, 2, 3, 4],
    "C":["one", "two", "three", "four"]
})
dfb

A,C
i64,str
1,"""one"""
2,"""two"""
3,"""three"""
4,"""four"""


In [26]:
dfa.join(dfb, how="left", on="A")

A,B,C
i64,str,str
1,"""uno""","""one"""
3,"""tres""","""three"""
4,"""cuatro""","""four"""
3,"""tres""","""three"""
2,"""dos""","""two"""


In [ ]:
# PASAR A RADIANES
# HIVERSINE DISTANCE
# JOIN CON LOS COUNTIES MAS CERCANOS